In [1]:
import sys, json
sys.path.append('../freebase_api')
import GstoreConnector
import datalib
import unidecode
import tqdm

In [2]:
def load_entities(data):
    entities = []

    for i in range(len(data)):
        for j in range(len(data[i]['entities'])):
            if data[i]['entities'][j][0] not in entities:
                entities.append(data[i]['entities'][j][0])
                
    return entities

def load_answers(data):
    answers = []
    
    for i in range(len(data)):
        for j in range(len(data[i]['answers'])):
            if data[i]['answers'][j][0] not in answers:
                answers.append(unidecode.unidecode(data[i]['answers'][j]))
    
    return answers

def sparql_filter(labels):
        """ generate sparql WHERE{} sub-query that keeps just the values in labels """

        value_filter = ['STRLANG("' + l.lower() + '", "en")' for l in labels]
        sparql_filter = '''FILTER(LCASE(?value) IN (''' + ', '.join(value_filter) + '''))'''
        return sparql_filter

In [3]:
main_data = []
with open('../data/main/trainmodel.json', 'r') as f:
    main_data = json.load(f)

In [4]:
answers = load_answers(main_data)

In [5]:
answers[40]

'New Moon'

In [3]:
#set the access username and password
username = "endpoint"
password = "123"

In [4]:
# start a gStoreConnector with given IP and Port
gc = GstoreConnector.GstoreConnector("freebase.gstore-pku.com", 80)

In [9]:
mids = {}

In [10]:
for answer in tqdm.tqdm(answers):
    sparql = """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                select ?s where 
                { 
                    ?s ?rel "%s"@en . 
                }
                LIMIT 1""" %(answer)
    # do query (you will get the result in json format)
    res = json.loads(gc.query(username, password, "freebase", sparql))
    if 'results' in res.keys() and len(res['results']['bindings']) > 0:
        mids[answer] = res['results']['bindings'][0]['s']['value']

100%|██████████| 6668/6668 [37:19<00:00,  3.10it/s]  


In [11]:
with open('../data/answer_mids.json', 'w') as fp:
    json.dump(mids, fp)

In [5]:
mids = {}
with open('../data/answer_mids.json') as f:
    mids = json.load(f)

In [6]:
for name,mid in tqdm.tqdm(mids.items()):
    sparql = """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?val WHERE {
        <%s> ?rel ?val .
        MINUS {
             <%s> <http://www.w3.org/2000/01/rdf-schema#label> ?val .

           }
    } LIMIT 500""" %(mid,mid)

    # do query (you will get the result in json format)
    entities = []
    
    res = json.loads(gc.query(username, password, "freebase", sparql))
    
    if 'results' in res.keys() and len(res['results']['bindings']) > 0:
        for b in res['results']['bindings']:
            if 'freebase' in b['val']['value']:
                entities.append(b['val']['value'])
    
    if len(name) < 200:
        with open('../data/answer-entities/{0}.txt'.format(name.replace("/","--")),'w') as f:
            for e in entities:
                f.write("{0}\n".format(e))

100%|██████████| 4351/4351 [1:07:39<00:00,  1.06s/it]


#### Для каждого answer делаем запрос на сущность, получаем ИД, и по этому ИД делаем запрос на связанные сущности за исключением лейбла. Полученные данные сохраняем в отдельный файл для отдельного answer. Затем, загружаем answers как матрицу и засовываем в BOW/TF-IDF